## Analysing the data from the Market Research Survey

In [1]:
# import libraries
import pandas as pd

In [2]:
# import csv file as dataframe
data_path = "tudublin_amenities_access_survey.csv"
survey_data = pd.read_csv(data_path, delimiter=",", encoding='unicode_escape')
survey_data.head(5)

,Id,Start time,Completion time,Email,Name,Which county do you work/live in?,Which sector do you work in?\n,Does your job involve accessing/working with data on publicly available amenities?\n,Which devices do you primarily use for your work?\n,What type of public amenities information do you usually require for your job?\n,...,Do you have any additional suggestions/feedback on our proposal ?\n,"If you'd like to contact us for more information on the project, please share your email below\n",Which devices do you primarily use for your day-to-day personal tasks?\n,How often do you use digital tools/applications for navigation?\n,"Based on the demo below, how useful would a web app be for accessing parking locations and quantities in your day-to-day?\n",Why would this be impractical for you?\n,Please tell us what you are using instead\n,What other public amenity would you like more information on?,What other features would you like to see alongside location & quantity of parking spaces?\n,"If you'd like to contact us for more information on the project, please share your email below\n1"
0,1,22/10/2024 18:14,22/10/2024 18:17,anonymous,NaN,NaN,Education,Yes,Desktop computer;Laptop;Smartphone;,"Mechanical (water grid, electric grid, etc);Tr...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,29/10/2024 16:29,29/10/2024 16:36,anonymous,NaN,Dublin,Government,No,NaN,NaN,...,NaN,NaN,Desktop computer;Laptop;Smartphone;,Weekly,Extremely useful,NaN,NaN,NaN,Cost;Parking hours;,NaN
2,3,31/10/2024 12:18,31/10/2024 12:19,anonymous,NaN,Fingal,Construction,No,NaN,NaN,...,NaN,NaN,Desktop computer;,Never,Somewhat useful,NaN,NaN,NaN,Parking hours;,NaN
3,4,31/10/2024 12:25,31/10/2024 12:29,anonymous,NaN,Dublin,Architecture,Yes,Desktop computer;Laptop;Smartphone;,"Recreational (parks, sport facilities, hiking ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,31/10/2024 12:24,31/10/2024 12:29,anonymous,NaN,Dublin,Construction,Yes,Desktop computer;,"Healthcare & Safety (emergency services, hospi...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# check number of rows and columns
print("Number of columns: ", len(survey_data.columns))
print("Number of rows: ", len(survey_data))

Number of columns:  36
Number of rows:  111


In [4]:
# check data type of columns -- all object
print("Data types of columns:")
survey_data.dtypes

Data types of columns:


Id                                                                                                                               int64
Start time                                                                                                                      object
Completion time                                                                                                                 object
Email                                                                                                                           object
Name                                                                                                                           float64
Which county do you work/live in?                                                                                               object
Which sector do you work in?\n                                                                                                  object
Does your job involve accessing/working with data on pu

### First order of business: clean up the df
<li> I will start by removing the unecessary columns: start & completion time, name & email because they're empty.<br></li>
<li>Need to rename the columns as well</li>
<li>Next, I will fill in the NAN in county responses, and group the different "other" responses in job<br></li>
<li>Then, I will split the df into those who are employed and unemployed, and then those who use public amenities or not in their professional life.</li>

In [5]:
# remove unecessary columns
survey_data = survey_data.drop(survey_data.columns[[1,2,3,4]], axis=1)
survey_data.head(5)

,Id,Which county do you work/live in?,Which sector do you work in?\n,Does your job involve accessing/working with data on publicly available amenities?\n,Which devices do you primarily use for your work?\n,What type of public amenities information do you usually require for your job?\n,How often do you require/access this information?,What type of software/tools do you use to access information on publicly available amenities?\n,How often do you use this software/tool?,"In regards to the public amenities chosen, does your current software/tool satisfy your work requirements?\n",...,Do you have any additional suggestions/feedback on our proposal ?\n,"If you'd like to contact us for more information on the project, please share your email below\n",Which devices do you primarily use for your day-to-day personal tasks?\n,How often do you use digital tools/applications for navigation?\n,"Based on the demo below, how useful would a web app be for accessing parking locations and quantities in your day-to-day?\n",Why would this be impractical for you?\n,Please tell us what you are using instead\n,What other public amenity would you like more information on?,What other features would you like to see alongside location & quantity of parking spaces?\n,"If you'd like to contact us for more information on the project, please share your email below\n1"
0,1,NaN,Education,Yes,Desktop computer;Laptop;Smartphone;,"Mechanical (water grid, electric grid, etc);Tr...",Weekly,Government database (i.e: data.gov.ie);,Monthly,Somewhat,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Dublin,Government,No,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Desktop computer;Laptop;Smartphone;,Weekly,Extremely useful,NaN,NaN,NaN,Cost;Parking hours;,NaN
2,3,Fingal,Construction,No,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Desktop computer;,Never,Somewhat useful,NaN,NaN,NaN,Parking hours;,NaN
3,4,Dublin,Architecture,Yes,Desktop computer;Laptop;Smartphone;,"Recreational (parks, sport facilities, hiking ...",Daily,Navigation applications (i.e: Google Maps);,Daily,Somewhat,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Dublin,Construction,Yes,Desktop computer;,"Healthcare & Safety (emergency services, hospi...",Weekly,Navigation applications (i.e: Google Maps);Cit...,Weekly,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# import column names
col_file = open("col_names.txt", "r")
col_names = col_file.read()
col_list = col_names.replace(' ','').split(",")
print(col_list)
col_file.close()

['id', 'county', 'sector', 'use_amenity_data', 'device_work', 'type_amenity_data_work', 'freq_amenity_work', 'type_tool_work', 'freq_tool_work', 'satisfaction_tool_work', 'why_unsatisfied_tool_work', 'feature_wish_work', 'demo_useful_work', 'why_impractical_demo_work', 'other_amenity_work', 'searchfunctionality_useful_rank', 'realtimeavail_useful_rank', 'qtyamenity_useful_rank', 'costamenity_useful_rank', 'routeplan_useful_rank', 'filtertype_useful_rank', 'export_useful_rank', 'feedback_work', 'contact_work', 'device_personal', 'freq_tool_personal', 'demo_useful_personal', 'why_impractical_demo_personal', 'other_tool_personal', 'other_amenity_personal', 'other_feature_personal', 'contact_personal']


In [12]:
# rename columns
survey_data = survey_data.set_axis(col_list, axis=1)
print(survey_data.columns)

Index(['id', 'county', 'sector', 'use_amenity_data', 'device_work',
       'type_amenity_data_work', 'freq_amenity_work', 'type_tool_work',
       'freq_tool_work', 'satisfaction_tool_work', 'why_unsatisfied_tool_work',
       'feature_wish_work', 'demo_useful_work', 'why_impractical_demo_work',
       'other_amenity_work', 'searchfunctionality_useful_rank',
       'realtimeavail_useful_rank', 'qtyamenity_useful_rank',
       'costamenity_useful_rank', 'routeplan_useful_rank',
       'filtertype_useful_rank', 'export_useful_rank', 'feedback_work',
       'contact_work', 'device_personal', 'freq_tool_personal',
       'demo_useful_personal', 'why_impractical_demo_personal',
       'other_tool_personal', 'other_amenity_personal',
       'other_feature_personal', 'contact_personal'],
      dtype='object')


In [13]:
survey_data.head(5)

,id,county,sector,use_amenity_data,device_work,type_amenity_data_work,freq_amenity_work,type_tool_work,freq_tool_work,satisfaction_tool_work,...,feedback_work,contact_work,device_personal,freq_tool_personal,demo_useful_personal,why_impractical_demo_personal,other_tool_personal,other_amenity_personal,other_feature_personal,contact_personal
0,1,NaN,Education,Yes,Desktop computer;Laptop;Smartphone;,"Mechanical (water grid, electric grid, etc);Tr...",Weekly,Government database (i.e: data.gov.ie);,Monthly,Somewhat,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Dublin,Government,No,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Desktop computer;Laptop;Smartphone;,Weekly,Extremely useful,NaN,NaN,NaN,Cost;Parking hours;,NaN
2,3,Fingal,Construction,No,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Desktop computer;,Never,Somewhat useful,NaN,NaN,NaN,Parking hours;,NaN
3,4,Dublin,Architecture,Yes,Desktop computer;Laptop;Smartphone;,"Recreational (parks, sport facilities, hiking ...",Daily,Navigation applications (i.e: Google Maps);,Daily,Somewhat,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Dublin,Construction,Yes,Desktop computer;,"Healthcare & Safety (emergency services, hospi...",Weekly,Navigation applications (i.e: Google Maps);Cit...,Weekly,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# split in 2 sub dataframe, those that use amenity data for work and those who dont

### Second order of business: probably encoding some of the columns